# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

# Solución de raspado web

## Paso 1: Instalar dependencias

Para comenzar, debemos instalar las librerías necesarias para el proyecto.  
Usaremos `pandas`, `requests` y `lxml` para realizar el scraping y procesar los datos.

Ejecuta el siguiente comando en la celda de Python 👇


In [ ]:
!pip install pandas requests lxml

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns



## Paso 2: Descargar HTML

En este paso descargaremos el contenido HTML de la página web que vamos a analizar.

Utilizaremos la librería `requests` para enviar una solicitud HTTP al sitio web de **Wikipedia** que contiene la tabla de las canciones más reproducidas en Spotify.  
El objetivo es obtener el código fuente (HTML) y almacenarlo en una variable para procesarlo más adelante.


In [ ]:
import requests

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Canciones_m%C3%A1s_reproducidas_en_Spotify"

# Realizamos la solicitud HTTP
response = requests.get(url)

# Guardamos el contenido HTML de la respuesta
html = response.text

# Mostramos los primeros 500 caracteres para verificar
print(html[:500])


## Paso 3: Transformar el HTML y extraer la tabla

En este paso convertiremos el HTML descargado en una tabla utilizable.  
Usaremos `pandas.read_html` (que internamente usa `lxml`) para detectar tablas dentro del HTML y cargarlas como `DataFrame`.

**Objetivos:**
- Detectar y seleccionar la tabla principal de canciones.
- Revisar sus columnas originales.
- Renombrar columnas a nombres homogéneos.
- Convertir la columna de reproducciones a un valor numérico.
- (Opcional) Extraer el año de la fecha de lanzamiento.


In [ ]:
import pandas as pd
import re

# 1) Extraer todas las tablas del HTML (variable `html` creada en el Paso 2)
tables = pd.read_html(html)
print(f"Tablas encontradas: {len(tables)}")

# 2) Elegimos la primera tabla (ajústalo si tu impresión muestra otra estructura)
df = tables[0].copy()

print("Columnas originales:")
print(df.columns.tolist())

# 3) Normalizamos nombres de columnas (pueden variar según la versión de Wikipedia)
rename_map = {
    "Posición": "Posición",
    "Position": "Posición",
    "Canción": "Canción",
    "Canción​": "Canción",
    "Título": "Canción",
    "Song": "Canción",
    "Artista": "Artista",
    "Artista(s)": "Artista",
    "Artist(s)": "Artista",
    "Reproducciones": "Reproducciones",
    "Reproducciones (miles de millones)": "Reproducciones",
    "Streams (billions)": "Reproducciones",
    "Fecha de lanzamiento": "Fecha",
    "Año de lanzamiento": "Fecha",
    "Release date": "Fecha",
    "Year": "Fecha"
}
df = df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

# 4) Limpiar filas vacías o encabezados duplicados
df = df.dropna(how="all").reset_index(drop=True)

# 5) Convertir "Reproducciones" a número absoluto
#    La tabla a veces usa miles de millones (ej. "5,12" o "5.12"), otras veces número completo con separadores.
def parse_streams(x):
    s = str(x).strip()

    # Caso A: formato en miles de millones (5,12 o 5.12)
    if re.match(r"^\d{1,2}([.,]\d+)?$", s):
        s = s.replace(",", ".")
        try:
            return float(s) * 1_000_000_000
        except:
            return None

    # Caso B: número grande con separadores y texto extra
    s = re.sub(r"[^\d]", "", s)  # deja solo dígitos
    return float(s) if s.isdigit() else None

if "Reproducciones" in df.columns:
    df["Reproducciones_num"] = df["Reproducciones"].apply(parse_streams)

# 6) Extraer año (si la columna Fecha trae una fecha completa)
def extract_year(x):
    m = re.search(r"(19|20)\d{2}", str(x))
    return int(m.group()) if m else None

if "Fecha" in df.columns:
    df["Año"] = df["Fecha"].apply(extract_year)

# 7) Conservar columnas clave si existen
cols = [c for c in ["Posición","Canción","Artista","Reproducciones","Reproducciones_num","Fecha","Año"] if c in df.columns]
df = df[cols]

# 8) Ordenar por reproducciones (si se pudo parsear)
if "Reproducciones_num" in df.columns:
    df = df.sort_values("Reproducciones_num", ascending=False).reset_index(drop=True)

# Vista previa
df.head(10)
